### Importing Modules Required for Analysis.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from pandas_profiling import ProfileReport

In [2]:
df = pd.read_csv('2021_Competition_Training.csv')
pd.set_option("display.max_columns", None)
#raw_data.shape
df.head()

C:\Users\vasus\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (2,8,9,11,13,16,20,26,28,29,31,33,51,55,58,62,64,66,68,75,85,102,124,127,131,132,135,160,174,180,187,192,202,209,210,211,215,220,230,234,240,243,247,251,255,261,285,293,297,300,305,306,309,323,334,344,345,352,353,355,359) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,ID,auth_3mth_post_acute_dia,rx_gpi2_72_pmpm_cost_6to9m_b4,atlas_pct_laccess_child15,atlas_recfacpth14,atlas_pct_fmrkt_frveg16,atlas_pct_free_lunch14,bh_ip_snf_net_paid_pmpm_cost_9to12m_b4,auth_3mth_acute_ckd,bh_ncal_pmpm_ct,src_div_id,total_bh_copay_pmpm_cost_t_9-6-3m_b4,bh_ip_snf_net_paid_pmpm_cost_3to6m_b4,cons_chmi,mcc_ano_pmpm_ct_t_9-6-3m_b4,auth_3mth_post_acute_trm,rx_maint_pmpm_cost_t_12-9-6m_b4,auth_3mth_post_acute_rsk,cons_ltmedicr,rx_gpi4_6110_pmpm_ct,atlas_pc_snapben15,credit_bal_nonmtgcredit_60dpd,rx_bh_mbr_resp_pmpm_cost_9to12m_b4,rx_nonbh_pmpm_cost_t_9-6-3m_b4,atlas_pct_laccess_nhna15,auth_3mth_acute_vco,credit_hh_nonmtgcredit_60dpd,rx_bh_pmpm_ct_0to3m_b4,auth_3mth_dc_ltac,cons_lwcm10,auth_3mth_post_acute_inj,atlas_fsrpth14,auth_3mth_dc_home,atlas_wicspth12,rx_gpi2_17_pmpm_cost_t_12-9-6m_b4,cons_hxmioc,rx_generic_pmpm_cost_t_6-3-0m_b4,cmsd2_sns_digest_abdomen_pmpm_ct,atlas_ghveg_farms12,credit_hh_bankcardcredit_60dpd,total_outpatient_allowed_pmpm_cost_6to9m_b4,cons_cwht,atlas_netmigrationrate1016,atlas_pct_laccess_snap15,bh_ncdm_ind,rx_nonmaint_mbr_resp_pmpm_cost_9to12m_b4,atlas_retirement_destination_2015_upda,rx_overall_mbr_resp_pmpm_cost_t_6-3-0m_b4,atlas_naturalchangerate1016,ccsp_236_pct,bh_ip_snf_mbr_resp_pmpm_cost_6to9m_b4,rx_overall_dist_gpi6_pmpm_ct_t_6-3-0m_b4,auth_3mth_post_acute_ben,atlas_pct_laccess_hisp15,auth_3mth_dc_no_ref,rx_overall_mbr_resp_pmpm_cost,rx_overall_gpi_pmpm_ct_0to3m_b4,auth_3mth_dc_snf,rx_phar_cat_humana_pmpm_ct_t_9-6-3m_b4,atlas_pct_laccess_hhnv15,auth_3mth_acute_ccs_048,bh_ip_snf_net_paid_pmpm_cost_0to3m_b4,auth_3mth_acute_end,auth_3mth_psychic,atlas_hiamenity,auth_3mth_bh_acute,credit_bal_consumerfinance,auth_3mth_acute_chf,rx_overall_gpi_pmpm_ct_t_6-3-0m_b4,rwjf_uninsured_pct,mcc_chf_pmpm_ct_t_9-6-3m_b4,rx_mail_mbr_resp_pmpm_cost_0to3m_b4,bh_urgent_care_copay_pmpm_cost_t_12-9-6m_b4,auth_3mth_hospice,auth_3mth_acute_bld,atlas_pct_wic15,ccsp_193_pct,auth_3mth_dc_hospice,auth_3mth_acute_ccs_030,atlas_pct_fmrkt_baked16,rx_nonmaint_mbr_resp_pmpm_cost,auth_3mth_acute_skn,atlas_veg_farms12,atlas_vlfoodsec_13_15,rx_gpi2_34_dist_gpi6_pmpm_ct,bh_ip_snf_net_paid_pmpm_cost,credit_hh_bankcard_severederog,rx_hum_16_pmpm_ct,est_age,rx_maint_pmpm_cost_t_6-3-0m_b4,cnt_cp_webstatement_pmpm_ct,atlas_pct_laccess_seniors15,phy_em_px_pct,atlas_percapitainc,rwjf_uninsured_adults_pct,rx_generic_mbr_resp_pmpm_cost_0to3m_b4,auth_3mth_acute_neo,rwjf_air_pollute_density,rx_gpi2_02_pmpm_cost,atlas_recfac14,cons_mobplus,lab_albumin_loinc_pmpm_ct,atlas_pct_obese_adults13,rx_maint_net_paid_pmpm_cost_t_12-9-6m_b4,rev_pm_obsrm_pmpm_ct,atlas_pct_sfsp15,total_physician_office_net_paid_pmpm_cost_9to12m_b4,atlas_pc_dirsales12,med_ip_snf_admit_days_pmpm,rej_med_outpatient_visit_ct_pmpm_t_6-3-0m_b4,auth_3mth_post_acute_vco,cms_tot_partd_payment_amt,rx_nonotc_dist_gpi6_pmpm_ct,rx_nonmaint_pmpm_ct,rx_nonbh_mbr_resp_pmpm_cost_6to9m_b4,cons_stlnindx,atlas_hipov_1115,auth_3mth_post_acute_dig,rx_nonbh_mbr_resp_pmpm_cost,atlas_redemp_snaps16,atlas_berry_farms12,rej_med_ip_snf_coins_pmpm_cost_t_9-6-3m_b4,rwjf_inactivity_pct,rx_gpi2_72_pmpm_ct_6to9m_b4,cons_n2pmr,med_physician_office_allowed_pmpm_cost_t_9-6-3m_b4,auth_3mth_acute_res,rev_cms_ct_pmpm_ct,atlas_foodhub16,total_physician_office_copay_pmpm_cost,auth_3mth_acute_dig,auth_3mth_dc_acute_rehab,atlas_pct_fmrkt_anmlprod16,auth_3mth_post_acute_hdz,bh_ip_snf_mbr_resp_pmpm_cost_3to6m_b4,auth_3mth_acute_ccs_172,credit_num_agencyfirstmtg,total_physician_office_net_paid_pmpm_cost_t_9-6-3m_b4,auth_3mth_acute_ccs_154,atlas_type_2015_mining_no,atlas_agritrsm_rct12,rx_days_since_last_script,atlas_pct_laccess_pop15,auth_3mth_post_acute_res,auth_3mth_acute_inf,rx_gpi2_01_pmpm_cost_0to3m_b4,atlas_povertyallagespct,covid_vaccination,rwjf_uninsured_child_pct,rx_branded_pmpm_ct_t_6-3-0m_b4,med_outpatient_deduct_pmpm_cost_t_9-6-3m_b4,credit_bal_mtgcredit_new,atlas_low_employment_2015_update,atlas_pct_diabetes_adults13,atlas_pct_laccess_nhasian15,atlas_deep_pov_all,atlas_net_inte

### Data Cleaning.

In [3]:
#Droping column unnamed.
df = df.drop(['ID','Unnamed: 0'], axis = 1)
df.shape

(974842, 366)

In [4]:
#Refering to final grouping excel sheet dropping columns.
use_columns = pd.read_excel('Final_Groupings_Oct 3.xlsx', usecols = ['ID','Use in Model'])
drop_columns = use_columns[use_columns['Use in Model'] == "No"]["ID"].tolist()

df_1 = df.drop(drop_columns, axis = 1)
df_1.shape

(974842, 233)

In [5]:
#Regex operation to remove non-ascii values.
for col in df_1.columns:
    df_1[col].replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True) 

In [6]:
#Identifying rows with * value.
star_row_index = []
for col in df_1.columns:
    star_row_index.extend((df_1[df_1[col] == '*'].index.to_list()))

index_drop = list(set(star_row_index))

In [7]:
df_2 = df_1.drop(index = index_drop)
df_2.shape[0]/df_1.shape[0] #0.9970908106134122

0.9970908106134122

In [8]:
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
            " columns that have missing values.")
    return mis_val_table_ren_columns

missing_values_table(df_2)

Your selected dataframe has 233 columns.
There are 103 columns that have missing values.


,Missing Values,% of Total Values
hedis_dia_hba1c_ge9,311618,32.1
cons_ltmedicr,311312,32.0
cons_chmi,199190,20.5
cons_lwcm10,198795,20.5
cons_cwht,198746,20.4
...,...,...
credit_bal_bankcard_severederog,4608,0.5
atlas_netmigrationrate1016,4608,0.5
atlas_povertyallagespct,4608,0.5
atlas_net_international_migration_rate,4606,0.5


In [9]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 972006 entries, 0 to 974841
Columns: 233 entries, atlas_pct_laccess_child15 to race_cd
dtypes: float64(160), int64(12), object(61)
memory usage: 1.7+ GB


In [10]:
def impute_mean(X):
    return df_2[X].fillna(value=df_2[X].mean(), inplace=True)

In [11]:
impute_mean('credit_bal_autobank')
impute_mean('credit_bal_autofinance_new')
impute_mean('credit_bal_bankcard_severederog')
impute_mean('credit_bal_consumerfinance')
impute_mean('credit_bal_mtgcredit_new')
impute_mean('credit_bal_nonmtgcredit_60dpd')
impute_mean('credit_hh_1stmtgcredit')
impute_mean('credit_hh_bankcard_severederog')
impute_mean('credit_hh_bankcardcredit_60dpd')
impute_mean('credit_hh_nonmtgcredit_60dpd')
impute_mean('credit_num_agencyfirstmtg')
impute_mean('credit_num_autobank_new')
impute_mean('credit_num_autofinance')
impute_mean('credit_num_consumerfinance_new')
impute_mean('credit_num_nonmtgcredit_60dpd')
impute_mean('atlas_medhhinc')
impute_mean('atlas_naturalchangerate1016')
impute_mean('atlas_net_international_migration_rate')
impute_mean('atlas_netmigrationrate1016')
impute_mean('atlas_pct_laccess_black15')
impute_mean('atlas_pct_laccess_child15')
impute_mean('atlas_pct_laccess_hhnv15')
impute_mean('atlas_pct_laccess_hisp15')
impute_mean('atlas_pct_laccess_lowi15')
impute_mean('atlas_pct_laccess_multir15')
impute_mean('atlas_pct_laccess_nhasian15')
impute_mean('atlas_pct_laccess_nhna15')
impute_mean('atlas_pct_laccess_nhpi15')
impute_mean('atlas_pct_laccess_pop15')
impute_mean('atlas_pct_laccess_seniors15')
impute_mean('atlas_pct_laccess_snap15')
impute_mean('atlas_pct_laccess_white15')
impute_mean('atlas_pct_nslp15')
impute_mean('atlas_pct_obese_adults13')
impute_mean('atlas_pct_reduced_lunch14')
impute_mean('atlas_pct_sbp15')
impute_mean('atlas_pct_sfsp15')
impute_mean('atlas_povertyunder18pct')
impute_mean('cms_partd_ra_factor_amt')            
impute_mean('cms_risk_adjustment_factor_a_amt')
impute_mean('cms_tot_partd_payment_amt')
impute_mean('cons_cgqs')
impute_mean('cons_chmi')
impute_mean('cons_chva')
impute_mean('cons_cwht')
impute_mean('cons_lwcm07')
impute_mean('cons_lwcm10')
impute_mean('cons_nwperadult')
impute_mean('rwjf_men_hlth_prov_ratio')
impute_mean('rwjf_mental_distress_pct')
impute_mean('rwjf_mv_deaths_rate')
impute_mean('rwjf_resident_seg_black_inx')
impute_mean('rwjf_social_associate_rate')
impute_mean('rwjf_uninsured_adults_pct')
impute_mean('rwjf_uninsured_child_pct')
impute_mean('rwjf_uninsured_pct')
impute_mean('total_med_allowed_pmpm_cost_9to12m_b4')
impute_mean('total_outpatient_allowed_pmpm_cost_6to9m_b4')
impute_mean('total_outpatient_mbr_resp_pmpm_cost_6to9m_b4')
impute_mean('total_physician_office_net_paid_pmpm_cost_9to12m_b4')
impute_mean('cons_estinv30_rc')
impute_mean('atlas_pct_diabetes_adults13')
impute_mean('atlas_povertyallagespct')


In [12]:
missing_values_table(df_2)

Your selected dataframe has 233 columns.
There are 40 columns that have missing values.


,Missing Values,% of Total Values
hedis_dia_hba1c_ge9,311618,32.1
cons_ltmedicr,311312,32.0
cons_n2pmr,198695,20.4
cons_rxadhm,198689,20.4
cons_hhcomp,198685,20.4
cons_n2phi,198678,20.4
cons_hxwearbl,198676,20.4
cons_stlnindx,198673,20.4
cons_rxadhs,198668,20.4
cons_n2pwh,198663,20.4


In [13]:
def impute_mode(X):
    return df_2[X].fillna(value=df_2[X].mode()[0], inplace=True)

In [14]:
impute_mode('atlas_convspth14')
impute_mode('atlas_dirsales_farms12')
impute_mode('atlas_foodhub16')
impute_mode('atlas_foodinsec_13_15')
impute_mode('atlas_foodinsec_child_03_11')
impute_mode('atlas_pct_cacfp15')
impute_mode('atlas_pct_fmrkt_anmlprod16')
impute_mode('atlas_pct_free_lunch14')
impute_mode('atlas_recfac14')
impute_mode('atlas_redemp_snaps16')
impute_mode('atlas_type_2015_update')
impute_mode('atlas_vlfoodsec_13_15')
impute_mode('cons_n2phi')
impute_mode('cons_n2pmr')
impute_mode('cons_n2pwh')
impute_mode('rwjf_air_pollute_density')
impute_mode('rwjf_inactivity_pct')
impute_mode('rwjf_income_inequ_ratio')

In [15]:
missing_values_table(df_2)

Your selected dataframe has 233 columns.
There are 22 columns that have missing values.


,Missing Values,% of Total Values
hedis_dia_hba1c_ge9,311618,32.1
cons_ltmedicr,311312,32.0
cons_rxadhm,198689,20.4
cons_hhcomp,198685,20.4
cons_hxwearbl,198676,20.4
cons_stlnindx,198673,20.4
cons_rxadhs,198668,20.4
cons_rxmaint,198647,20.4
cons_hxmioc,198636,20.4
cms_orig_reas_entitle_cd,34456,3.5


In [16]:
def Category_X(X):
    return df_2[X].fillna('Category_X',inplace= True)

In [17]:
Category_X('atlas_farm_to_school13')
Category_X('hedis_dia_hba1c_ge9')
Category_X('atlas_hiamenity')
Category_X('atlas_hipov_1115')
Category_X('atlas_low_education_2015_update')
Category_X('atlas_low_employment_2015_update')
Category_X('atlas_perpov_1980_0711')
Category_X('atlas_persistentchildpoverty_1980_2011')
Category_X('atlas_population_loss_2015_update')
Category_X('atlas_retirement_destination_2015_upda')
Category_X('atlas_type_2015_mining_no')
Category_X('atlas_type_2015_recreation_no')
Category_X('cms_orig_reas_entitle_cd')
Category_X('cons_hxmioc')
Category_X('cons_hxwearbl')
Category_X('cons_ltmedicr')
Category_X('cons_rxadhm')
Category_X('cons_rxadhs')
Category_X('cons_rxmaint')
Category_X('cons_stlnindx')

In [18]:
missing_values_table(df_2)

Your selected dataframe has 233 columns.
There are 2 columns that have missing values.


,Missing Values,% of Total Values
cons_hhcomp,198685,20.4
race_cd,15683,1.6


In [19]:
#Replace race with 1.
df_2['race_cd'] = df_2['race_cd'].replace(np.nan,1)
df_2['race_cd'].unique()

array([1., 5., 0., 2., 3., 4., 6.])

In [20]:
#Replace race with U.
df_2['cons_hhcomp'] = df_2['cons_hhcomp'].replace(np.nan,'U')
df_2['cons_hhcomp'].unique()

array(['B', 'A', 'L', 'I', 'G', 'D', 'C', 'J', 'U', 'K', 'H', 'F', 'E'],
      dtype=object)

In [21]:
categorical_bolean = (df_2.dtypes == "object").values
training_categorical = df_2.loc[:,categorical_bolean]

In [22]:
counter = 1
for col in training_categorical.columns:
    print(f"{counter} {col} --> {training_categorical[col].unique()}")
    counter += 1

1 total_bh_copay_pmpm_cost_t_9-6-3m_b4 --> ['No Activity' 'No_Change' 'New' 'Resolved' 'Dec_2x-4x' 'Inc_over_8x'
 'Dec_1x-2x' 'Dec_over_8x' 'Inc_1x-2x' 'Inc_2x-4x' 'Inc_4x-8x' 'Dec_4x-8x']
2 mcc_ano_pmpm_ct_t_9-6-3m_b4 --> ['No Activity' 'Resolved' 'Inc_1x-2x' 'No_Change' 'New' 'Dec_2x-4x'
 'Dec_1x-2x']
3 rx_maint_pmpm_cost_t_12-9-6m_b4 --> ['Inc_1x-2x' 'No Activity' 'Dec_2x-4x' 'Dec_1x-2x' 'Inc_4x-8x' 'New'
 'Dec_4x-8x' 'Inc_2x-4x' 'Dec_over_8x' 'Resolved' 'No_Change'
 'Inc_over_8x']
4 cons_ltmedicr --> [4.0 3.0 'Category_X' 8.0 7.0 6.0 9.0 5.0 1.0 2.0 0.0]
5 rx_nonbh_pmpm_cost_t_9-6-3m_b4 --> ['Dec_1x-2x' 'New' 'Inc_2x-4x' 'Dec_over_8x' 'Dec_2x-4x' 'Inc_1x-2x'
 'No Activity' 'Inc_over_8x' 'Dec_4x-8x' 'No_Change' 'Inc_4x-8x'
 'Resolved']
6 rx_bh_pmpm_ct_0to3m_b4 --> ['0.0' '0.6666667' '1.0' '0.33333334' '1.3333334' '1.6666666' '2.3333333'
 '2.0' '2.6666667' '4.6666665' '3.6666667' '3.0' '0.5' '4.0' '5.3333335'
 '3.3333333' '6.6666665' '6.3333335' '5.0' '1.5' '6.0' '5.6666665' '2.5'
 '

In [23]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 972006 entries, 0 to 974841
Columns: 233 entries, atlas_pct_laccess_child15 to race_cd
dtypes: float64(141), int64(12), object(80)
memory usage: 1.7+ GB


In [24]:
#Changing 11 columns to numerics as currently reflected as object

df_2['rx_bh_pmpm_ct_0to3m_b4'] = pd.to_numeric(df_2['rx_bh_pmpm_ct_0to3m_b4'])
df_2['rx_gpi2_34_dist_gpi6_pmpm_ct'] = pd.to_numeric(df_2['rx_gpi2_34_dist_gpi6_pmpm_ct'])
df_2['lab_albumin_loinc_pmpm_ct'] = pd.to_numeric(df_2['lab_albumin_loinc_pmpm_ct'])
df_2['rx_gpi2_72_pmpm_ct_6to9m_b4'] = pd.to_numeric(df_2['rx_gpi2_72_pmpm_ct_6to9m_b4'])
df_2['auth_3mth_home'] = pd.to_numeric(df_2['auth_3mth_home']) 
df_2['rev_cms_ansth_pmpm_ct'] = pd.to_numeric(df_2['rev_cms_ansth_pmpm_ct']) 
df_2['auth_3mth_acute'] = pd.to_numeric(df_2['auth_3mth_acute']) 
df_2['rx_gpi2_90_dist_gpi6_pmpm_ct_9to12m_b4'] = pd.to_numeric(df_2['rx_gpi2_90_dist_gpi6_pmpm_ct_9to12m_b4']) 
df_2['rej_total_physician_office_visit_ct_pmpm_0to3m_b4'] = pd.to_numeric(df_2['rej_total_physician_office_visit_ct_pmpm_0to3m_b4']) 
df_2['rx_gpi2_56_dist_gpi6_pmpm_ct_3to6m_b4'] = pd.to_numeric(df_2['rx_gpi2_56_dist_gpi6_pmpm_ct_3to6m_b4']) 
df_2['rx_gpi2_33_pmpm_ct_0to3m_b4'] = pd.to_numeric(df_2['rx_gpi2_33_pmpm_ct_0to3m_b4'])

In [25]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 972006 entries, 0 to 974841
Columns: 233 entries, atlas_pct_laccess_child15 to race_cd
dtypes: float64(150), int64(14), object(69)
memory usage: 1.7+ GB


In [26]:
#Zero missing values 
missing_values_table(df_2)

Your selected dataframe has 233 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


profile = ProfileReport(df_2, title="Clean_EDA", minimal = True)
profile.to_file("Clean_EDA_Oct3.html")

In [27]:
cat_x_list = ["atlas_farm_to_school13","atlas_hiamenity","atlas_hipov_1115","atlas_low_education_2015_update","atlas_low_employment_2015_update",
"atlas_perpov_1980_0711","atlas_persistentchildpoverty_1980_2011","atlas_population_loss_2015_update","atlas_retirement_destination_2015_upda",
"atlas_type_2015_mining_no","atlas_type_2015_recreation_no","cms_orig_reas_entitle_cd",
"cons_hxmioc","cons_hxwearbl","cons_ltmedicr","cons_rxadhm","cons_rxadhs","cons_rxmaint","cons_stlnindx"]

counter = 0
for col in cat_x_list:
    print(f"{counter} {col} --> {df_2[col].unique()} --> {df_2[col].dtypes}")
    counter += 1    

0 atlas_farm_to_school13 --> [1.0 0.0 'Category_X'] --> object
1 atlas_hiamenity --> [0.0 1.0 'Category_X'] --> object
2 atlas_hipov_1115 --> [0.0 1.0 'Category_X'] --> object
3 atlas_low_education_2015_update --> [0.0 1.0 'Category_X'] --> object
4 atlas_low_employment_2015_update --> [0.0 1.0 'Category_X'] --> object
5 atlas_perpov_1980_0711 --> [0.0 'Category_X' 1.0] --> object
6 atlas_persistentchildpoverty_1980_2011 --> [0.0 1.0 'Category_X'] --> object
7 atlas_population_loss_2015_update --> [1.0 0.0 'Category_X'] --> object
8 atlas_retirement_destination_2015_upda --> [0.0 1.0 'Category_X'] --> object
9 atlas_type_2015_mining_no --> [0.0 'Category_X' 1.0] --> object
10 atlas_type_2015_recreation_no --> [0.0 'Category_X' 1.0] --> object
11 cms_orig_reas_entitle_cd --> [0.0 1.0 'Category_X' 3.0 2.0] --> object
12 cons_hxmioc --> [9.0 3.0 6.0 8.0 7.0 'Category_X' 4.0 2.0 1.0 5.0 0.0] --> object
13 cons_hxwearbl --> [8.0 9.0 2.0 6.0 7.0 'Category_X' 5.0 4.0 3.0 1.0 0.0] --> object
1

In [28]:
counter = 0
for col in cat_x_list:
    df_2[col] = df_2[col].astype('string')
    print(f"{counter} {col} --> {df_2[col].unique()} --> {df_2[col].dtypes}")
    counter += 1 

0 atlas_farm_to_school13 --> <StringArray>
['1.0', '0.0', 'Category_X']
Length: 3, dtype: string --> string
1 atlas_hiamenity --> <StringArray>
['0.0', '1.0', 'Category_X']
Length: 3, dtype: string --> string
2 atlas_hipov_1115 --> <StringArray>
['0.0', '1.0', 'Category_X']
Length: 3, dtype: string --> string
3 atlas_low_education_2015_update --> <StringArray>
['0.0', '1.0', 'Category_X']
Length: 3, dtype: string --> string
4 atlas_low_employment_2015_update --> <StringArray>
['0.0', '1.0', 'Category_X']
Length: 3, dtype: string --> string
5 atlas_perpov_1980_0711 --> <StringArray>
['0.0', 'Category_X', '1.0']
Length: 3, dtype: string --> string
6 atlas_persistentchildpoverty_1980_2011 --> <StringArray>
['0.0', '1.0', 'Category_X']
Length: 3, dtype: string --> string
7 atlas_population_loss_2015_update --> <StringArray>
['1.0', '0.0', 'Category_X']
Length: 3, dtype: string --> string
8 atlas_retirement_destination_2015_upda --> <StringArray>
['0.0', '1.0', 'Category_X']
Length: 3, dtyp

In [29]:
counter = 0
for col in cat_x_list:
    df_2[col] = df_2[col].astype(str)
    print(f"{counter} {col} --> {df_2[col].unique()} --> {df_2[col].dtypes}")
    counter += 1 

0 atlas_farm_to_school13 --> ['1.0' '0.0' 'Category_X'] --> object
1 atlas_hiamenity --> ['0.0' '1.0' 'Category_X'] --> object
2 atlas_hipov_1115 --> ['0.0' '1.0' 'Category_X'] --> object
3 atlas_low_education_2015_update --> ['0.0' '1.0' 'Category_X'] --> object
4 atlas_low_employment_2015_update --> ['0.0' '1.0' 'Category_X'] --> object
5 atlas_perpov_1980_0711 --> ['0.0' 'Category_X' '1.0'] --> object
6 atlas_persistentchildpoverty_1980_2011 --> ['0.0' '1.0' 'Category_X'] --> object
7 atlas_population_loss_2015_update --> ['1.0' '0.0' 'Category_X'] --> object
8 atlas_retirement_destination_2015_upda --> ['0.0' '1.0' 'Category_X'] --> object
9 atlas_type_2015_mining_no --> ['0.0' 'Category_X' '1.0'] --> object
10 atlas_type_2015_recreation_no --> ['0.0' 'Category_X' '1.0'] --> object
11 cms_orig_reas_entitle_cd --> ['0.0' '1.0' 'Category_X' '3.0' '2.0'] --> object
12 cons_hxmioc --> ['9.0' '3.0' '6.0' '8.0' '7.0' 'Category_X' '4.0' '2.0' '1.0' '5.0' '0.0'] --> object
13 cons_hxwearbl

# Preparing Test and Training Data

In [30]:
#clean_data = raw_data_2.dropna()
X = df_2.drop('covid_vaccination',axis=1)
y = df_2['covid_vaccination']

In [31]:
y.unique()

array(['no_vacc', 'vacc'], dtype=object)

In [32]:
y = y.replace(['vacc','no_vacc'], [1,0])
y

0         0
1         0
2         0
3         0
4         0
         ..
974837    0
974838    0
974839    0
974840    0
974841    0
Name: covid_vaccination, Length: 972006, dtype: int64

In [33]:
df_2.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 972006 entries, 0 to 974841
Data columns (total 233 columns):
 #    Column                                                 Dtype  
---   ------                                                 -----  
 0    atlas_pct_laccess_child15                              float64
 1    atlas_pct_free_lunch14                                 float64
 2    total_bh_copay_pmpm_cost_t_9-6-3m_b4                   object 
 3    cons_chmi                                              float64
 4    mcc_ano_pmpm_ct_t_9-6-3m_b4                            object 
 5    rx_maint_pmpm_cost_t_12-9-6m_b4                        object 
 6    cons_ltmedicr                                          object 
 7    credit_bal_nonmtgcredit_60dpd                          float64
 8    rx_bh_mbr_resp_pmpm_cost_9to12m_b4                     float64
 9    rx_nonbh_pmpm_cost_t_9-6-3m_b4                         object 
 10   atlas_pct_laccess_nhna15                              

In [34]:
categorical_bolean = (X.dtypes == "object").values
training_categorical = X.loc[:,categorical_bolean]
training_numerical = X.loc[:,~categorical_bolean]

In [35]:
categorical_column = list(training_categorical)

In [36]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()

X[categorical_column] = enc.fit_transform(X[categorical_column])

In [37]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)

In [38]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler


num_attribs = list(training_numerical)
cat_attribs = list(training_categorical)


full_pipeline = ColumnTransformer([
        ("std_scaler", StandardScaler(), num_attribs)
], remainder='passthrough')

X_train_prepared = full_pipeline.fit_transform(X_train)

In [56]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


cv = KFold(n_splits=4, random_state=1, shuffle=True)
lr_model = LogisticRegression(max_iter=10000)

scores = cross_val_score(lr_model, X_train_prepared, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)

In [57]:
from numpy import mean
from numpy import std

print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Accuracy: 0.657 (0.001)


In [59]:
lr_model.fit(X_train_prepared, y_train)

LogisticRegression(max_iter=10000)

In [61]:
X_test_prepared = full_pipeline.transform(X_test)

In [63]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test, lr_model.predict_proba(X_test_prepared)[:, 1])

0.658650999090179